### Importing Libraries

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

In [66]:
# By contatenating both training and testint dataset we ensure that
# all the feature engineering and preprocesseing is mantained in both
# datasets.  
df_train = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('./datasets/test.csv')
df = pd.concat([df_train, df_test], ignore_index=True)
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500.0
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000.0
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000.0
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000.0
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500.0


In [67]:
df_test.shape

(878, 80)

In [68]:
df.describe()['Total Bsmt SF']

count    2928.000000
mean     1050.901981
std       438.999166
min         0.000000
25%       793.000000
50%       990.000000
75%      1300.500000
max      6110.000000
Name: Total Bsmt SF, dtype: float64

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2929 entries, 0 to 2928
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2929 non-null   int64  
 1   PID              2929 non-null   int64  
 2   MS SubClass      2929 non-null   int64  
 3   MS Zoning        2929 non-null   object 
 4   Lot Frontage     2439 non-null   float64
 5   Lot Area         2929 non-null   int64  
 6   Street           2929 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2929 non-null   object 
 9   Land Contour     2929 non-null   object 
 10  Utilities        2929 non-null   object 
 11  Lot Config       2929 non-null   object 
 12  Land Slope       2929 non-null   object 
 13  Neighborhood     2929 non-null   object 
 14  Condition 1      2929 non-null   object 
 15  Condition 2      2929 non-null   object 
 16  Bldg Type        2929 non-null   object 
 17  House Style   

### Data Cleaning and Feature Engineering

In [70]:
# droping columns based on missing nans
# print(f'Cols before droping nan\'s: {df.shape[1]}')
# temp = df.dropna(axis='columns')
# print(f'Cols after droping nan\'s: {temp.shape[1]}')
# df = temp

In [71]:
df.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

In [72]:
df.corr()['SalePrice'].sort_values()

PID               -0.255052
Enclosed Porch    -0.135656
Kitchen AbvGr     -0.125444
Overall Cond      -0.097019
MS SubClass       -0.087335
Id                -0.051398
Bsmt Half Bath    -0.045328
Low Qual Fin SF   -0.041594
Yr Sold           -0.015203
Misc Val          -0.007375
BsmtFin SF 2       0.016255
Pool Area          0.023106
Mo Sold            0.032735
3Ssn Porch         0.048732
Screen Porch       0.134581
Bedroom AbvGr      0.137067
Bsmt Unf SF        0.190210
2nd Flr SF         0.248452
Half Bath          0.283001
Bsmt Full Bath     0.283662
Lot Area           0.296566
Wood Deck SF       0.326490
Open Porch SF      0.333476
Lot Frontage       0.341842
BsmtFin SF 1       0.423519
Fireplaces         0.471093
TotRms AbvGrd      0.504014
Mas Vnr Area       0.512230
Garage Yr Blt      0.533922
Full Bath          0.537969
Year Remod/Add     0.550370
Year Built         0.571849
1st Flr SF         0.618486
Total Bsmt SF      0.628925
Garage Cars        0.648220
Garage Area        0

In [73]:
# Selecting top 5 Features with high corr
features_high_corr = ['Overall Qual', 'Gr Liv Area', 'Garage Area', 'Total Bsmt SF', '1st Flr SF', 'SalePrice']
df_first_iter = df[features_high_corr]
df[features_high_corr].dtypes

Overall Qual       int64
Gr Liv Area        int64
Garage Area      float64
Total Bsmt SF    float64
1st Flr SF         int64
SalePrice        float64
dtype: object

In [74]:
df_first_iter.head()

,Overall Qual,Gr Liv Area,Garage Area,Total Bsmt SF,1st Flr SF,SalePrice
0,6,1479,475.0,725.0,725,130500.0
1,7,2122,559.0,913.0,913,220000.0
2,5,1057,246.0,1057.0,1057,109000.0
3,5,1444,400.0,384.0,744,174000.0
4,6,1445,484.0,676.0,831,138500.0


In [75]:
df_first_iter.isna().sum()

Overall Qual       0
Gr Liv Area        0
Garage Area        1
Total Bsmt SF      1
1st Flr SF         0
SalePrice        878
dtype: int64

In [76]:
df_first_iter.shape

(2929, 6)

### Sub-linear regression model to calculate missing values

In [77]:
proc_train = df_first_iter.loc[df['SalePrice'].notna()]
proc_test = df_first_iter.loc[df['SalePrice'].isna()]

In [78]:
proc_train.corr()['Garage Area'].sort_values()

Total Bsmt SF    0.488805
Gr Liv Area      0.490949
1st Flr SF       0.498712
Overall Qual     0.563814
SalePrice        0.650270
Garage Area      1.000000
Name: Garage Area, dtype: float64

In [79]:
#Getting only high correlated data points
series_GA_corr = proc_train.corr()['Garage Area'] >= .5
features_high_corr = series_GA_corr[series_GA_corr].index.values
df_ga = proc_train[features_high_corr]
df_ga.dtypes

Overall Qual      int64
Garage Area     float64
SalePrice       float64
dtype: object

In [80]:
series_bs_corr = proc_train.corr()['Total Bsmt SF'] >= .5
features_high_corr = series_bs_corr[series_bs_corr].index.values
df_bsmt_sf = proc_train[features_high_corr]
df_bsmt_sf.dtypes

Overall Qual       int64
Total Bsmt SF    float64
1st Flr SF         int64
SalePrice        float64
dtype: object

In [81]:
proc_train.isna().sum()

Overall Qual     0
Gr Liv Area      0
Garage Area      1
Total Bsmt SF    1
1st Flr SF       0
SalePrice        0
dtype: int64

In [82]:
proc_test.isna().sum()

Overall Qual       0
Gr Liv Area        0
Garage Area        0
Total Bsmt SF      0
1st Flr SF         0
SalePrice        878
dtype: int64

In [83]:
# Imputing feature Garage Area

In [84]:
X_sub_1 = proc_train[df_ga.columns]
X_sub_1 = X_sub_1.drop(columns='Garage Area')

y_sub_1 = proc_train['Garage Area']

print(X_sub_1.shape)
print(y_sub_1.shape)

(2051, 2)
(2051,)


In [87]:
lr_1 = LinearRegression()

X_test_sub1 = proc_train.loc[df['Garage Area'].isna()]
X_test_sub1 = X_test_sub1[df_ga.columns]
X_test_sub1 = X_test_sub1.drop(columns=['Garage Area'])

print(X_test_sub1.shape)

X_train, X_test, y_train, y_test = train_test_split(X_sub_1, y_sub_1, random_state = 30)
lr_1.fit(X_train, y_train)

preds_sub1 = lr_1.predict(X_test_sub1)

(1, 2)


In [88]:
preds_sub1

array([409.26541105])

In [89]:
proc_train.loc[df['Garage Area'].isna()]['Garage Area']

1712   NaN
Name: Garage Area, dtype: float64

In [90]:
proc_train.at[1712, 'Garage Area'] = preds_sub1[0] 

In [91]:
proc_train.isna().sum()

Overall Qual     0
Gr Liv Area      0
Garage Area      0
Total Bsmt SF    1
1st Flr SF       0
SalePrice        0
dtype: int64

In [22]:
# imputing feature Total Bsmt SF

In [93]:
X_sub_2 = proc_train.loc[df['Total Bsmt SF'].notna()]
X_sub_2 = X_sub_2[df_bsmt_sf.columns]
X_sub_2 = X_sub_2.drop(columns=['Total Bsmt SF'])

#y_sub_2 = proc_train['Total Bsmt SF']
y_sub_2 = proc_train.loc[df['Total Bsmt SF'].notna()]
y_sub_2 = y_sub_2['Total Bsmt SF']

print(X_sub_2.shape)
print(y_sub_2.shape)

(2050, 3)
(2050,)


In [95]:
lr_2 = LinearRegression()

X_test_sub2 = proc_train.loc[df['Total Bsmt SF'].isna()]
X_test_sub2 = X_test_sub2[df_bsmt_sf.columns]
X_test_sub2 = X_test_sub2.drop(columns=['Total Bsmt SF'])
print(X_test_sub2.shape)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_sub_2, y_sub_2, random_state = 30)
lr_2.fit(X_train_2, y_train_2)

preds_sub2 = lr_2.predict(X_test_sub2)
preds_sub2

(1, 3)


array([702.42721611])

In [96]:
proc_train.loc[df['Total Bsmt SF'].isna()]['Total Bsmt SF']

1327   NaN
Name: Total Bsmt SF, dtype: float64

In [97]:
proc_train.at[1327, 'Total Bsmt SF'] = preds_sub2[0] 

### Define feature Matrix and Target

In [98]:
proc_train.isna().sum()

Overall Qual     0
Gr Liv Area      0
Garage Area      0
Total Bsmt SF    0
1st Flr SF       0
SalePrice        0
dtype: int64

In [99]:
proc_test.isna().sum()

Overall Qual       0
Gr Liv Area        0
Garage Area        0
Total Bsmt SF      0
1st Flr SF         0
SalePrice        878
dtype: int64

In [100]:
X = proc_train.drop(columns=['SalePrice'])
y = proc_train['SalePrice']

In [101]:
ss = StandardScaler()
ss.fit(X)
X_scaled = ss.transform(X)

In [102]:
poly = PolynomialFeatures(include_bias=False)

In [103]:
X_poly = poly.fit_transform(X_scaled)

### Instantiate, Fit and Model 

In [104]:
lr = LinearRegression()

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state = 30)

In [106]:
lr.fit(X_train, y_train)

LinearRegression()

In [107]:
lr.score(X_train, y_train), lr.score(X_test, y_test)

(0.8806149341879038, 0.4406182217211265)

In [52]:
cross_val_score(lr, X_poly, y, cv=5).mean()

0.849333494566277

### Run model through test.csv

In [27]:
X_TEST = proc_test.drop(columns=['SalePrice'])

In [28]:
poly_TEST = PolynomialFeatures(include_bias=False)
X_TEST_poly  = poly_TEST.fit_transform(X_TEST)

In [29]:
preds = lr.predict(X_TEST_poly)

In [30]:
df_test['y_hat'] = preds

In [31]:
df.iloc[2051,:]

Id                   2658
PID             902301120
MS SubClass           190
MS Zoning              RM
Lot Frontage           69
                  ...    
Misc Val                0
Mo Sold                 4
Yr Sold              2006
Sale Type             WD 
SalePrice             NaN
Name: 2051, Length: 81, dtype: object

In [32]:
proc_test.head()

,Overall Qual,Gr Liv Area,Garage Area,Total Bsmt SF,1st Flr SF,SalePrice
2051,6,1928,440.0,1020.0,908,NaN
2052,5,1967,580.0,1967.0,1967,NaN
2053,7,1496,426.0,654.0,664,NaN
2054,5,968,480.0,968.0,968,NaN
2055,6,1394,514.0,1394.0,1394,NaN


In [33]:
df_test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,y_hat
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,201561.621465
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,8,2006,WD,161954.967057
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,9,2006,New,161197.989577
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2007,WD,127276.149492
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,185,0,NaN,NaN,NaN,0,7,2009,WD,185546.145123


In [34]:
df_submission = df_test[['Id','y_hat']]

In [35]:
df_submission = df_submission.rename(columns={'y_hat':'SalePrice'})

In [36]:
df_submission

,Id,SalePrice
0,2658,201561.621465
1,2718,161954.967057
2,2414,161197.989577
3,1989,127276.149492
4,625,185546.145123
...,...,...
873,1662,197121.051965
874,1234,203510.380315
875,1373,135870.476143
876,1672,104443.118376


### Export as CSV

In [37]:
df_submission.to_csv('my_submission_v2.csv', index=False)